In [27]:
import pandas as pd
import os
import numpy as np

In [28]:
data1 = pd.read_csv('pulsar_stars.csv')
data1.head()

,Mean of the integrated profile,Standard deviation of the integrated profile,Excess kurtosis of the integrated profile,Skewness of the integrated profile,Mean of the DM-SNR curve,Standard deviation of the DM-SNR curve,Excess kurtosis of the DM-SNR curve,Skewness of the DM-SNR curve,target_class
0,140.562500,55.683782,-0.234571,-0.699648,3.199833,19.110426,7.975532,74.242225,0
1,102.507812,58.882430,0.465318,-0.515088,1.677258,14.860146,10.576487,127.393580,0
2,103.015625,39.341649,0.323328,1.051164,3.121237,21.744669,7.735822,63.171909,0
3,136.750000,57.178449,-0.068415,-0.636238,3.642977,20.959280,6.896499,53.593661,0
4,88.726562,40.672225,0.600866,1.123492,1.178930,11.468720,14.269573,252.567306,0


In [29]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17898 entries, 0 to 17897
Data columns (total 9 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0    Mean of the integrated profile                17898 non-null  float64
 1    Standard deviation of the integrated profile  17898 non-null  float64
 2    Excess kurtosis of the integrated profile     17898 non-null  float64
 3    Skewness of the integrated profile            17898 non-null  float64
 4    Mean of the DM-SNR curve                      17898 non-null  float64
 5    Standard deviation of the DM-SNR curve        17898 non-null  float64
 6    Excess kurtosis of the DM-SNR curve           17898 non-null  float64
 7    Skewness of the DM-SNR curve                  17898 non-null  float64
 8   target_class                                   17898 non-null  int64  
dtypes: float64(8), int64(1)
memory usage: 1.2 MB


In [30]:
data1["target_class"].value_counts()

0    16259
1     1639
Name: target_class, dtype: int64

In [31]:
# Stratified Sampling using Scikit-learn's Stratified Shuffle Split Class
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.25, random_state=42)
for train_index, test_index in split.split(data1, data1["target_class"]):
    strat_train_set = data1.loc[train_index]
    strat_test_set = data1.loc[test_index]

In [32]:
 strat_test_set["target_class"].value_counts()

0    4065
1     410
Name: target_class, dtype: int64

In [33]:
strat_train_set["target_class"].value_counts()

0    12194
1     1229
Name: target_class, dtype: int64

In [34]:
train_set = strat_train_set.drop("target_class", axis=1) # drop labels for training set
train_labels = strat_train_set["target_class"].copy()
test_set = strat_test_set.drop("target_class", axis=1) # drop labels for testing set
test_labels = strat_test_set["target_class"].copy()

In [35]:
train_labels

15026    0
14795    0
5940     0
3002     0
12431    0
        ..
6349     0
12584    0
15650    0
4596     0
5560     1
Name: target_class, Length: 13423, dtype: int64

In [36]:
test_labels

1261     0
2554     0
9121     0
8051     1
7877     0
        ..
11379    1
14231    0
15239    0
7661     0
1282     0
Name: target_class, Length: 4475, dtype: int64

In [37]:
!pip install fuzzy-c-means
from fcmeans import FCM
tr_set=train_set.T
fcm = FCM(n_clusters=4,random_state=42)
fcm.fit(tr_set.to_numpy())
tr_fin  = fcm.centers.T

In [38]:
te_set=test_set.T
fcm = FCM(n_clusters=4, random_state=42)
fcm.fit(te_set.to_numpy())
te_fin  = fcm.centers.T

In [39]:
tr_fin.shape

(13423, 4)

In [40]:
train_labels.shape

(13423,)

In [41]:
from sklearn.metrics import matthews_corrcoef
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
parameters = {"loss" : ["hinge", "log", "squared_hinge", "modified_huber", "perceptron"],
              "alpha" : [0.0001, 0.001, 0.01, 0.1],
              "penalty" : ["l2", "l1", "elasticnet", "none"],
              "tol": [1.e-3]
             }

clf2 = SGDClassifier(early_stopping= True, max_iter=10000, random_state=42)
clf = GridSearchCV(clf2, parameters, scoring = 'balanced_accuracy', cv=10)
res1=clf.fit(tr_fin, train_labels)
res1.best_estimator_

SGDClassifier(alpha=0.1, average=False, class_weight=None, early_stopping=True,
              epsilon=0.1, eta0=0.0, fit_intercept=True, l1_ratio=0.15,
              learning_rate='optimal', loss='squared_hinge', max_iter=10000,
              n_iter_no_change=5, n_jobs=None, penalty='elasticnet',
              power_t=0.5, random_state=42, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [42]:
res1.best_params_

{'alpha': 0.1, 'loss': 'squared_hinge', 'penalty': 'elasticnet', 'tol': 0.001}

In [43]:
res1.best_score_

0.893669273335238

In [44]:
y_test_pred=res1.best_estimator_.predict(te_fin)


In [45]:
import sklearn
from sklearn.metrics import accuracy_score
sklearn.metrics.accuracy_score(test_labels, y_test_pred)*100

97.00558659217877

In [46]:
#error rate=100-accuracy
100-(sklearn.metrics.accuracy_score(test_labels, y_test_pred)*100)

2.9944134078212272

In [47]:
matthews_corrcoef(test_labels, y_test_pred)*100

80.70200131393335

In [48]:
from sklearn.metrics import precision_score
sklearn.metrics.precision_score(test_labels, y_test_pred, pos_label= 1)*100

98.93617021276596

In [49]:
from sklearn.metrics import recall_score
sklearn.metrics.recall_score(test_labels, y_test_pred, pos_label= 1)*100

68.04878048780488

In [50]:
from sklearn.metrics import f1_score
sklearn.metrics.f1_score(test_labels, y_test_pred,  pos_label= 1)*100

80.63583815028902